# BCI AVM Hypertuning
### Training the Machine Learning Models on Tabular Data: 

This notebook covers the following steps:
- Import data from AWS
- Visualize the data using Seaborn and matplotlib
- Run a parallel hyperparameter sweep to train machine learning models on the dataset
- Explore the results of the hyperparameter sweep with MLflow
- Register the best performing model in MLflow

## Requirements
This notebook requires Databricks Runtime for Machine Learning or a similar spark/pyspark enabled environment setup locally (or elsewhere).  
If you are using Databricks Runtime 7.3 LTS ML or below, you must update the CloudPickle library using the commands in the following cell.

In [2]:
import cloudpickle

## Importing Data
  
In this section, you download a dataset from the web (AWS S3 Bucket).

1. Ensure that you have installed `bciavm` using `pip install bciavm` in your local machine or on Databricks.

In [4]:
import io
from bciavm.core.config import your_bucket
from bciavm.utils.bci_utils import ReadParquetFile, get_postcodeOutcode_from_postcode, get_postcodeArea_from_outcode, drop_outliers, preprocess_data
import pandas as pd
import bciavm
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
import sys
import hyperopt
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope
from math import exp
import mlflow
import os
import gc
from bciavm.pipelines import RegressionPipeline
import numpy as np

2021-06-15 01:07:28,134 featuretools - WARNING Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.

In [5]:
MLFLOW_TRACKING_URI = os.environ["MLFLOW_TRACKING_URI"]
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/tune-hyperparameters')

INFO: '/Users/mike.casale@blockchainclimate.org/Experiments/tune-hyperparameters' does not exist. Creating a new experiment

In [6]:
dfPricesEpc = pd.DataFrame()
dfPrices = pd.DataFrame()

yearArray = ['2020', '2019']
for year in yearArray:
    singlePriceEpcFile = pd.DataFrame(ReadParquetFile(your_bucket, 'epc_price_data/byDate/2021-02-04/parquet/' + year))
    dfPricesEpc = dfPricesEpc.append(singlePriceEpcFile)

dfPricesEpc['POSTCODE_OUTCODE'] = dfPricesEpc['Postcode'].apply(get_postcodeOutcode_from_postcode)
dfPricesEpc['POSTCODE_AREA'] = dfPricesEpc['POSTCODE_OUTCODE'].apply(get_postcodeArea_from_outcode)
dfPricesEpc.groupby('TypeOfMatching_m').count()['Postcode']

Out[3]: TypeOfMatching_m
1. Address Matched 699206
2. Address Matched No Spec 26055
3. No in Address Matched 325243
4. No match 339476
Name: Postcode, dtype: int64

## Preprocessing Data
Prior to training a model, check for missing values and split the data into training and validation sets.

In [8]:
for n in range(100):
  try: 
    X_train, X_test, y_train, y_test = bciavm.utils.bci_utils.preprocess_data(
      dfPricesEpc.rename({'Postcode':'POSTCODE'},axis=1).sample(50000))
    break
  except: pass
  
X_train

Out[4]:

,unit_indx,POSTCODE,POSTCODE_OUTCODE,POSTTOWN_e,PROPERTY_TYPE_e,TOTAL_FLOOR_AREA_e,NUMBER_HEATED_ROOMS_e,FLOOR_LEVEL_e,Latitude_m,Longitude_m,POSTCODE_AREA
8917,27701,DL10 7DL,DL10,RICHMOND,House,87.00,5.0,0.0,54.407468,-1.718436,DL
1703,10829,CF83 3QE,CF83,CAERPHILLY,House,55.00,3.0,0.0,51.583956,-3.204553,CF
15156,16802,NG18 4YD,NG18,MANSFIELD,Flat,70.00,4.0,0.0,53.134176,-1.182214,NG
20813,18876,BH24 2LZ,BH24,RINGWOOD,Bungalow,279.00,8.0,0.0,50.839537,-1.837949,BH
10087,11472,SO40 8US,SO40,SOUTHAMPTON,Bungalow,56.06,3.0,0.0,50.921369,-1.514176,SO
...,...,...,...,...,...,...,...,...,...,...,...
11443,26899,BB11 4LX,BB11,BURNLEY,House,72.45,4.0,0.0,53.785870,-2.253289,BB
3437,21339,CB1 2NU,CB1,CAMBRIDGE,House,54.00,4.0,0.0,52.200276,0.140023,CB
8888,15256,LS7 4EF,LS7,LEEDS,House,170.00,5.0,0.0,53.815708,-1.530616,LS
17809,12897,TS25 5QQ,TS25,HARTLEPOOL,House,102.00,5.0,0.0,54.672936,-1.231791,TS


##Hypertuning the AVM pipeline

The following code uses the `xgboost` and `scikit-learn` libraries to train a valuation model. It runs a parallel hyperparameter sweep to train multiple
models in parallel, using Hyperopt and SparkTrials. The code tracks the performance of each parameter configuration with MLflow.

In [10]:
version_info = sys.version_info
PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                              minor=version_info.minor,
                                              micro=version_info.micro)


conda_env = {'channels': ['defaults','conda-forge'],
            'dependencies': [
                'python={}'.format(PYTHON_VERSION),
                'pip',
                  {'pip': ['bciavm==1.21.5',
                           'dask-ml',
                           'hyperopt'
                          ],
                  },
            ],
            'name': 'mlflow-env'
}

In [11]:
try: os.mkdir('/dbfs/FileStore/tables/avm/tuning/')
except: pass

In [12]:
search_space = {
  'stacking': hp.choice('stacking', [True, True, False]),
  
  #Transformer tuning
  'numeric_impute_strategy': hp.choice('numeric_impute_strategy', ["mean", "median", "most_frequent"]),
  'top_n': hp.choice('top_n', [1, 2, 3, 4, 5, 6]),
    
  #K nearest neighbor tuning
  'n_neighbors': hp.choice('n_neighbors', [1, 2, 3, 4, 5, 6, 7, 8, 9]),
  'leaf_size': hp.choice('leaf_size', [1, 2, 3, 4, 5, 6, 7]),
  'p': 2,
    
  #MultiLayer Perceptron Regressor tuning
  'activation': 'relu',
  'solver': 'adam',
  'batch_size': scope.int(hp.quniform('batch_size', 600, 700, 1)),
  'alpha': hp.choice('alpha', [0.02, 0.03]),
  'learning_rate_init': hp.choice('learning_rate_init', [0.005, 0.01, 0.1, 0.125, 0.133, 0.15]),
  'max_iter': hp.choice('max_iter', [150, 160, 170, 180, 190, 200]),
  'beta_1': hp.choice('beta_1', [0.2, 0.25, 0.3]),
  'epsilon': hp.choice('epsilon', [1e-08, 1e-07, 1e-09]),

  #XGBoost Regressor tuning
  'n_estimators': scope.int(hp.quniform('n_estimators', 400, 600, 10)),
  'max_depth': scope.int(hp.quniform('max_depth', 80, 200, 5)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.choice('min_child_weight', [5, 6, 7]),
  
  #Final Estimator XGBoost Regressor tuning
  'fe_n_estimators': scope.int(hp.quniform('fe_n_estimators', 100, 1000, 10)),
  'fe_max_depth': scope.int(hp.quniform('fe_max_depth', 4, 100, 1)),
  'fe_learning_rate': hp.loguniform('fe_learning_rate', -3, 0),
  'fe_reg_alpha': hp.choice('fe_reg_alpha', [0.005, 0.01, 0.02, 0.03, 0.04, 0.05]),
  'fe_reg_lambda': hp.choice('fe_reg_lambda', [0.005, 0.01, 0.02, 0.03, 0.04, 0.05]),
  'fe_min_child_weight': hp.choice('fe_min_child_weight', [1, 2]),
  'fe_metric':'mae',
  'fe_objective': 'reg:squarederror',
  'fe_seed': 123, # Set a seed for deterministic training
}

In [13]:
def train_model(params):

    mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/tune')
    
    # With MLflow autologging, hyperparameters and the trained model are automatically logged to MLflow.
    mlflow.sklearn.autolog()

    with mlflow.start_run(nested=True):
      
        if params['stacking']:
            class Pipeline(RegressionPipeline):
                custom_name = 'AVM Pipeline'
                component_graph = {  'Preprocess Transformer': [
                                         'Preprocess Transformer'
                                     ],
                                     'Imputer': [
                                         'Imputer', 
                                         'Preprocess Transformer'
                                     ],
                                     'One Hot Encoder': [
                                         'One Hot Encoder', 
                                         'Imputer'
                                     ],
                                     'K Nearest Neighbors': [
                                         'K Nearest Neighbors Regressor',
                                         'One Hot Encoder'
                                     ],
                                     'XGB Regressor': [
                                         'XGBoost Regressor', 
                                         'One Hot Encoder'
                                     ],
                                     'ML Perceptron Regressor': [
                                         'MultiLayer Perceptron Regressor', 
                                         'One Hot Encoder'
                                     ],
                                     'Linear Regressor Stack': [
                                         'Linear Regressor',
                                         'K Nearest Neighbors',
                                         'XGB Regressor'
                                     ],
                                     'Final Estimator': [
                                         'XGBoost Regressor', 
                                         'Linear Regressor Stack',
                                         'K Nearest Neighbors',
                                         'XGB Regressor',
                                         'ML Perceptron Regressor',
                                         'One Hot Encoder'
                                     ]}

            #Uses the best params from the Hypertuning Notebook
            #TODO: automate by reading best params from mlflow logged trials
            parameters = {'Imputer': {'categorical_impute_strategy': 'most_frequent',
                            'numeric_impute_strategy': params['numeric_impute_strategy'],
                            'categorical_fill_value': None,
                            'numeric_fill_value': None,
                          },
                         'One Hot Encoder': {'top_n': params['top_n'],
                            'features_to_encode': ['agg_cat'],
                            'categories': None,
                            'drop': None,
                            'handle_unknown': 'ignore',
                            'handle_missing': 'error',
                          },
                         'K Nearest Neighbors': {'n_neighbors': params['n_neighbors'],
                            'weights': 'distance',
                            'algorithm': 'auto',
                            'leaf_size': params['leaf_size'],
                            'p': params['p'],
                            'metric': 'minkowski',
                            'n_jobs': 1
                         },
                         'ML Perceptron Regressor': {'activation': 'relu',
                            'solver': 'adam',
                            'alpha': float(params['alpha']),
                            'batch_size': int(params['batch_size']),
                            'learning_rate': 'constant',
                            'learning_rate_init': float(params['learning_rate_init']),
                            'max_iter': int(params['max_iter']),
                            'early_stopping': True,
                            'beta_1': float(params['beta_1']),
                            'beta_2': 0.999,
                            'epsilon': float(params['epsilon']),
                            'n_iter_no_change': 10
                         },
                         'XGB Regressor': {'learning_rate': params['learning_rate'],
                                    'max_depth': params['max_depth'],
                                    'min_child_weight': params['min_child_weight'],
                                    'reg_alpha': params['reg_alpha'],
                                    'reg_lambda': params['reg_lambda'],
                                    'n_estimators': params['n_estimators']
                         },
                         'Final Estimator': {'learning_rate': params['fe_learning_rate'],
                                            'max_depth': params['fe_max_depth'],
                                            'min_child_weight': params['fe_min_child_weight'],
                                            'reg_alpha': params['fe_reg_alpha'],
                                            'reg_lambda': params['fe_reg_lambda'],
                                            'n_estimators': params['fe_n_estimators']
                         }
            }
        
        else:
            class Pipeline(RegressionPipeline):
                custom_name = 'AVM Pipeline'
                component_graph = {  'Preprocess Transformer': [
                                         'Preprocess Transformer'
                                     ],
                                     'Imputer': [
                                         'Imputer', 
                                         'Preprocess Transformer'
                                     ],
                                     'One Hot Encoder': [
                                         'One Hot Encoder', 
                                         'Imputer'
                                     ],
                                     'XGB Regressor': [
                                         'XGBoost Regressor', 
                                         'One Hot Encoder'
                                     ]}

            #Uses the best params from the Hypertuning Notebook
            #TODO: automate by reading best params from mlflow logged trials
            parameters = {'Imputer': {'categorical_impute_strategy': 'most_frequent',
                            'numeric_impute_strategy': params['numeric_impute_strategy'],
                            'categorical_fill_value': None,
                            'numeric_fill_value': None,
                          },
                         'One Hot Encoder': {'top_n': params['top_n'],
                            'features_to_encode': ['agg_cat'],
                            'categories': None,
                            'drop': None,
                            'handle_unknown': 'ignore',
                            'handle_missing': 'error',
                          },
                         'XGB Regressor': {'learning_rate': params['fe_learning_rate'],
                                    'max_depth': params['fe_max_depth'],
                                    'min_child_weight': params['fe_min_child_weight'],
                                    'reg_alpha': params['fe_reg_alpha'],
                                    'reg_lambda': params['fe_reg_lambda'],
                                    'n_estimators': params['fe_n_estimators']
                         }
            }
            
        for component in parameters:
          for param in parameters[component]:
            _component_param = component + '_' + param
            mlflow.log_param(_component_param, parameters[component][param])

        avm_pipeline = Pipeline(parameters=parameters)
        avm_pipeline.fit(X_train, y_train)

        # Compute and return trial error
        scores = avm_pipeline.score(X_test, 
                                         y_test, 
                                         objectives=['MAPE',
                                                   'MdAPE',
                                                   'ExpVariance',
                                                   'MaxError',
                                                   'MedianAE',
                                                   'MSE',
                                                   'MAE',
                                                   'R2',
                                                   'Root Mean Squared Error'])
        MdAPE = scores['MdAPE']

        #Examine the learned feature importances output by the model as a sanity-check.
        fi = pd.DataFrame({'feature':avm_pipeline.get_component("XGB Regressor").input_feature_names,'importance':avm_pipeline.get_component("XGB Regressor").feature_importance}).sort_values(by='importance', ascending=False)

        #Log the feature importances output as an artifact
        artifact_path = '/dbfs/FileStore/tables/avm/tuning/XGBoost_importance.csv'
        fi.to_csv(artifact_path,index=False)
        mlflow.log_artifact(artifact_path)

        #Log the scoring metrics
        mlflow.log_metric('MAPE', scores['MAPE'])
        mlflow.log_metric('MdAPE', scores['MdAPE'])
        mlflow.log_metric('ExpVariance', scores['ExpVariance'])
        mlflow.log_metric('MaxError', scores['MaxError'])
        mlflow.log_metric('MedianAE', scores['MedianAE'])
        mlflow.log_metric('MSE', scores['MSE'])
        mlflow.log_metric('MAE', scores['MAE'])
        mlflow.log_metric('R2', scores['R2'])
        mlflow.log_metric('Root Mean Squared Error', scores['Root Mean Squared Error'])

        #Log an input example for future reference
        input_example = X_train.dropna().sample(1)

        #Log the mlflow model, along with the conda environment and input example
        mlflow.sklearn.log_model(
                             avm_pipeline,
                             "avm", 
                             conda_env=conda_env,
                             input_example=input_example
                            )

        # fmin minimizes the MdAPE (median absolute percentage error)
        return {'status': STATUS_OK, 'loss': scores['MdAPE']}

# Greater parallelism will lead to speedups, but a less optimal hyperparameter sweep. 
# A reasonable value for parallelism is the square root of max_evals.
spark_trials = SparkTrials(parallelism=3)

# Run fmin within an MLflow run context so that each hyperparameter configuration is logged as a child run of a parent
mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/tune')
with mlflow.start_run(run_name='avm_tune'):
    best_params = fmin(
        fn=train_model, 
        space=search_space, 
        algo=tpe.suggest, 
        #max_evals=42,
        timeout=10800,
        trials=spark_trials,
        rstate=np.random.RandomState(123)
        )

0%| | 0/9223372036854775807 [00:00<?, ?trial/s, best loss=?] 0%| | 1/9223372036854775807 [08:49<1356590715622856676:41:36, 529.49s/trial, best loss: 16.068350409836064] 0%| | 2/9223372036854775807 [08:56<955003350732570769:38:08, 372.75s/trial, best loss: 16.068350409836064]  0%| | 3/9223372036854775807 [14:00<902370788160533372:01:04, 352.21s/trial, best loss: 16.06722892696348]  0%| | 4/9223372036854775807 [16:14<734747781162453638:15:28, 286.78s/trial, best loss: 16.06722892696348] 0%| | 5/9223372036854775807 [17:47<585872778041062559:17:20, 228.67s/trial, best loss: 16.06722892696348] 0%| | 6/9223372036854775807 [18:16<431661202658606885:32:48, 168.48s/trial, best loss: 16.06722892696348] 0%| | 7/9223372036854775807 [20:00<382171437137226483:29:04, 149.17s/trial, best loss: 16.06722892696348] 0%| | 8/9223372036854775807 [28:52<676813389938833344:17:04, 264.17s/trial, best loss: 16.06722892696348] 0%| | 9/9223372036854775807 [33:54<706104312993762800:04:16, 275.60s/trial, best loss: 16.06722892696348] 0%| | 10/9223372036854775807 [35:12<554286589700938428:52:16, 216.35s/trial, best loss: 11.860074513381996] 0%| | 11/9223372036854775807 [35:36<405700405289304209:38:08, 158.35s/trial, best loss: 11.860074513381996] 0%| | 12/9223372036854775807 [41:01<534020536644116225:08:16, 208.44s/trial, best loss: 11.860074513381996] 0%| | 13/9223372036854775807 [45:09<564602537284382811:01:20, 220.37s/trial, best loss: 10.889102193995381] 0%| | 14/9223372036854775807 [47:00<480622878324941273:18:56, 187.59s/trial, best loss: 10.889102193995381] 0%| | 15/9223372036854775807 [49:15<440305674521477120:00:00, 171.86s/trial, best loss: 10.889102193995381] 0%| | 16/9223372036854775807 [50:40<373607791264871947:22:40, 145.82s/trial, best loss: 10.889102193995381] 0%| | 17/9223372036854775807 [54:54<456169049010165805:30:40, 178.05s/trial, best loss: 10.889102193995381] 0%| | 18/9223372036854775807 [55:03<326263725142656869:15:44, 127.34s/trial, best loss: 10.889102193995381] 0%| | 19/9223372036854775807 [55:45<260716874014031580:43:44, 101.76s/trial, best loss: 10.889102193995381] 0%| | 20/9223372036854775807 [59:05<336397199428706704:29:52, 131.30s/trial, best loss: 10.889102193995381] 0%| | 21/9223372036854775807 [1:00:49<315511521405731471:21:36, 123.15s/trial, best loss: 10.889102193995381] 0%| | 22/9223372036854775807 [1:02:09<282434064335626567:40:48, 110.24s/trial, best loss: 10.762487499999999] 0%| | 23/9223372036854775807 [1:04:33<308512771834486747:35:28, 120.42s/trial, best loss: 10.762487499999999] 0%| | 24/9223372036854775807 [1:05:14<247530071606503851:48:16, 96.61s/trial, best loss: 10.762487499999999]  0%| | 25/9223372036854775807 [1:05:41<194064691505641590:19:44, 75.75s/trial, best loss: 10.762487499999999] 0%| | 26/9223372036854775807 [1:06:42<182142453150061982:09:04, 71.09s/trial, best loss: 10.762487499999999] 0%| | 27/9223372036854775807 [1:07:41<172918518530815135:17:20, 67.49s/trial, best loss: 10.762487499999999] 0%| | 28/9223372036854775807 [1:08:10<143377833965030149:41:20, 55.96s/trial, best loss: 10.762487499999999] 0%| | 29/9223372036854775807 [1:10:11<193475189461789973:37:04, 75.52s/trial, best loss: 10.762487499999999] 0%| | 30/9223372036854775807 [1:10:31<150845328785940225:08:16, 58.88s/trial, best loss: 10.762487499999999] 0%| | 31/9223372036854775807 [1:10:57<125618291563319041:08:16, 49.03s/trial, best loss: 10.762487499999999] 0%| | 32/9223372036854775807 [1:13:04<185665496756956328:23:28, 72.47s/trial, best loss: 10.762487499999999] 0%| | 33/9223372036854775807 [1:15:44<253093283343333931:14:08, 98.79s/trial, best loss: 10.762487499999999] 0%| | 34/9223372036854775807 [1:15:47<179499328979392921:36:00, 70.06s/trial, best loss: 10.762487499999999] 0%| | 35/9223372036854775807 [1:16:58<179530593112152692:03:12, 70.07s/trial, best loss: 10.762487499999999] 0%| | 36/9223372036854775807 [1:21:10<319596467909638590:00:32, 124.74s/trial, best loss: 10.762487499999999] 0%| | 

#### Use MLflow to view the results
Open up the Experiment Runs sidebar to see the MLflow runs. Click on Date next to the down arrow to display a menu, and select 'MdAPE' to display the runs sorted by the MdAPE metric. The lowest MdAPE value is ~10%. 

MLflow tracks the parameters and performance metrics of each run. Click the External Link icon <img src="https://docs.databricks.com/_static/images/external-link.png"/> at the top of the Experiment Runs sidebar to navigate to the MLflow Runs Table.

Now investigate how the hyperparameter choice correlates with MdAPE. Click the "+" icon to expand the parent run, then select all runs except the parent, and click "Compare". Select the Parallel Coordinates Plot.

The Parallel Coordinates Plot is useful in understanding the impact of parameters on a metric. You can drag the pink slider bar at the upper right corner of the plot to highlight a subset of MdAPE values and the corresponding parameter values. The plot below highlights the highest MdAPE values:

<img src="https://docs.databricks.com/_static/images/mlflow/end-to-end-example/parallel-coordinates-plot.png"/>

Notice that all of the top performing runs have a low value for reg_lambda and learning_rate. 

You could run another hyperparameter sweep to explore even lower values for these parameters. For simplicity, that step is not included here.

You used MLflow to log the model produced by each hyperparameter configuration. The following code finds the best performing run and saves the model to the model registry.

In [17]:
best_run = mlflow.search_runs(order_by=['metrics.MdAPE ASC']).iloc[0]
print(f'MdAPE of Best Run: {best_run["metrics.MdAPE"]}')

MdAPE of Best Run: 10.762487499999999